In [1]:
import os
import copy
import numpy as np
import librosa
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

In [36]:
def get_features(utterance: np.ndarray, sr: int, n_mfccs: int=20):

    mfccs  = librosa.feature.mfcc(utterance, sr, n_mfcc=n_mfccs)
    mfccs1 = librosa.feature.delta(mfccs)
    mfccs2 = librosa.feature.delta(mfccs, order=2)

    features = np.hstack((mfccs.T, mfccs1.T, mfccs2.T))
    return features

def get_features_from_directory(directory: str, ignore_files=['.DS_Store']):
    
    features = np.empty((0, 60))
    for filename in os.listdir(directory):
        if filename in ignore_files: continue

        utt, sr = librosa.load(os.path.join(directory, filename))
        utt = librosa.util.normalize(utt)
        features_utt = get_features(utt, sr)
        features = np.vstack((features, features_utt))
    
    return features

In [18]:
def map_adapt(ubm, X, max_iter=100, r=16):
    
    gmm = copy.deepcopy(ubm)
    
    for _ in range(max_iter):
        n = np.sum(gmm.predict_proba(X), axis=0).reshape(-1, 1) # (K, 1)
        X_tilde = (1 / n) * gmm.predict_proba(X).T.dot(X) # (K, F)
        alpha = (n / (n + r)).reshape(-1, 1) # (K, 1)
        gmm.means_ = alpha * X_tilde + (1 - alpha) * gmm.means_
    
    return gmm

In [68]:
def test_gmm(gmms: list[GaussianMixture],
             labels: list[str],
             correct_label: str,
             directory: str,
             ignore_files=['.DS_Store']):

    results = []
    for filepath in os.listdir(directory):
        if filepath in ignore_files: continue
        
        utt, sr = librosa.load(os.path.join(directory, filepath))
        utt = librosa.util.normalize(utt)
        features = get_features(utt, sr)

        prediction = []
        for gmm, label in zip(gmms, labels):
            prediction.append(gmm.score(features))

        results.append(labels[np.argmax(prediction)] == correct_label)

    results = np.array(results, dtype=int)

    return np.sum(results) / results.shape[0]

# Training

In [5]:
female_dir = './dataset_supervectors/train_data/UBM/female'
male_dir   = './dataset_supervectors/train_data/UBM/male'

In [6]:
features_ubm = np.empty((0, 60))
for filepath in os.listdir(female_dir):
    utt, sr = librosa.load(os.path.join(female_dir, filepath))
    utt = librosa.util.normalize(utt)
    features_utt = get_features(utt, sr)
    features_ubm = np.vstack((features_ubm, features_utt))

In [7]:
features_ubm.shape

(12954, 60)

In [8]:
for filepath in os.listdir(male_dir):
    utt, sr = librosa.load(os.path.join(male_dir, filepath))
    utt = librosa.util.normalize(utt)
    features_utt = get_features(utt, sr)
    features_ubm = np.vstack((features_ubm, features_utt))

In [66]:
features_ubm.shape

(25952, 60)

## Train Universal Background Model

In [75]:
ubm = GaussianMixture(n_components=32,
                      covariance_type='diag',
                      n_init=3,
                      max_iter=200)

ubm.fit(features_ubm)

GaussianMixture(covariance_type='diag', max_iter=200, n_components=32, n_init=3)

In [11]:
ubm.means_.shape

(128, 60)

## Train Female and Male models

$T$ — количество семплов, $F$ — количество признаков, $K$ — количество компонент в гауссовой смеси.

In [94]:
female_dir_train = './dataset_supervectors/train_data/female'
male_dir_train   = './dataset_supervectors/train_data/male'

# Relevance parameter.
r = 16

# Number of adaptation iterations.
max_iter = 10

### Get female model

In [13]:
features_female = get_features_from_directory(female_dir_train)

In [95]:
gmm_female = map_adapt(ubm, features_female, max_iter=max_iter, r=r)

### Get male model

In [24]:
features_male = get_features_from_directory(male_dir_train)

In [96]:
gmm_male = map_adapt(ubm, features_male, max_iter=max_iter, r=r)

### Get female model no UBM

In [88]:
gmm_female_no_ubm = GaussianMixture(n_components=64,
                                    covariance_type='diag',
                                    n_init=3,
                                    max_iter=200)

gmm_female_no_ubm.fit(features_female)

GaussianMixture(covariance_type='diag', max_iter=200, n_components=64, n_init=3)

### Get male model no UBM

In [89]:
gmm_male_no_ubm = GaussianMixture(n_components=64,
                                  covariance_type='diag',
                                  n_init=3,
                                  max_iter=200)

gmm_male_no_ubm.fit(features_male)

GaussianMixture(covariance_type='diag', max_iter=200, n_components=64, n_init=3)

# Evaluation

In [97]:
female_acc = test_gmm([gmm_female, gmm_male],
                      labels=['f', 'm'],
                      correct_label='f',
                      directory='./dataset_supervectors/test_data/female')

In [98]:
female_acc

0.8469387755102041

In [99]:
male_acc = test_gmm([gmm_female, gmm_male],
                    labels=['f', 'm'],
                    correct_label='m',
                    directory='./dataset_supervectors/test_data/male')

In [100]:
male_acc

0.7937062937062938

In [90]:
female_no_ubm_acc = test_gmm([gmm_female_no_ubm, gmm_male_no_ubm],
                             labels=['f', 'm'],
                             correct_label='f',
                             directory='./dataset_supervectors/test_data/female')

In [91]:
female_no_ubm_acc

0.8537414965986394

In [92]:
male_no_ubm_acc = test_gmm([gmm_female_no_ubm, gmm_male_no_ubm],
                           labels=['f', 'm'],
                           correct_label='m',
                           directory='./dataset_supervectors/test_data/male')

In [93]:
male_no_ubm_acc

0.8041958041958042